# train

In [1]:
import yaml
import os

with open("./config.yaml", 'rb') as f:
    config = yaml.load(f)

/home/hiroki/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  """


In [2]:
OUTPUT_ROOT = config['IO_OPTION']['OUTPUT_ROOT']
os.makedirs(OUTPUT_ROOT, exist_ok=True)

## load library

In [3]:
# python default library
import os
import shutil
import datetime
import sys
import pickle

# general analysis tool-kit
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

# pytorch
import torch
from torch import optim, nn
from torch.utils.tensorboard import SummaryWriter

from torchsummary import summary

# etc
import yaml
yaml.warnings({'YAMLLoadWarning': False})
import mlflow
from collections import defaultdict
from scipy.stats import zscore

# original library

import common as com
import pytorch_modeler as modeler
from pytorch_model import DAGMM as Model
from pytorch_utils import filtered_load_model
import models

import librosa
import IPython
import librosa.display

## load config and set logger

In [4]:
#with open("./config.yaml", 'rb') as f:
#    config = yaml.load(f)

log_folder = config['IO_OPTION']['OUTPUT_ROOT']+'/{0}.log'.format(datetime.date.today())
logger = com.setup_logger(log_folder, '00_train.py')

/home/hiroki/anaconda3/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## Setting

In [5]:
# Setting seed
modeler.set_seed(42)

In [6]:
############################################################################
# Setting I/O path
############################################################################
# input dirs
INPUT_ROOT = config['IO_OPTION']['INPUT_ROOT']
dev_path = INPUT_ROOT + "/dev_data"
add_dev_path = INPUT_ROOT + "/add_dev_data"
# machine type
MACHINE_TYPE = config['IO_OPTION']['MACHINE_TYPE']
machine_types = os.listdir(dev_path)
# output dirs
OUTPUT_ROOT = config['IO_OPTION']['OUTPUT_ROOT']
MODEL_DIR = config['IO_OPTION']['OUTPUT_ROOT'] + '/models'
TB_DIR = config['IO_OPTION']['OUTPUT_ROOT'] + '/tb'
PKL_DIR = OUTPUT_ROOT + '/pkl'
#os.makedirs(OUTPUT_ROOT, exist_ok=True)
os.makedirs(MODEL_DIR, exist_ok=True)
os.makedirs(TB_DIR, exist_ok=True)
os.makedirs(PKL_DIR, exist_ok=True)
# copy config
shutil.copy('./config.yaml', OUTPUT_ROOT)

'/media/hiroki/working/research/dcase2020/result/2D/DAGMM/v4/config.yaml'

## make path list and train/valid split

In [7]:
############################################################################
# make path set and train/valid split
############################################################################
'''
train_paths[machine_type]['train' or 'valid'] = path
'''
dev_train_paths = {}
add_train_paths = {}
train_paths = {}

In [8]:
for machine_type in machine_types:
    # dev train
    dev_train_paths = ["{}/{}/train/".format(dev_path, machine_type) + file for file in os.listdir("{}/{}/train".format(dev_path, machine_type))]
    dev_train_paths = sorted(dev_train_paths)
    # add_dev train
    add_train_paths = ["{}/{}/train/".format(add_dev_path, machine_type) + file for file in os.listdir("{}/{}/train".format(add_dev_path, machine_type))]
    add_train_paths = sorted(add_train_paths)
    # valid
    dev_valid_paths = ["{}/{}/test/".format(dev_path, machine_type) + file for file in os.listdir("{}/{}/test".format(dev_path, machine_type))]
    dev_valid_paths = sorted(dev_valid_paths)
    
    train_paths[machine_type] = {}
    train_paths[machine_type]['train'] = dev_train_paths + add_train_paths
    train_paths[machine_type]['valid'] = dev_valid_paths

## training

In [9]:
#############################################################################
# run
#############################################################################
def run(machine_type):
    com.tic()
    logger.info('TARGET MACHINE_TYPE: {0}'.format(machine_type))
    logger.info('MAKE DATA_LOADER')
    # dev_train_paths
    dataloaders_dict = modeler.make_dataloader(train_paths, machine_type)
    # define writer for tensorbord
    os.makedirs(TB_DIR+'/'+machine_type, exist_ok=True)         # debug
    tb_log_dir = TB_DIR + '/' + machine_type
    writer = SummaryWriter(log_dir = tb_log_dir)
    logger.info('TRAINING')
    # parameter setting
    net = Model(sample_rate=config['preprocessing']['sample_rate'],
                window_size=config['preprocessing']['window_size'],
                hop_size=config['preprocessing']['hop_size'],
                mel_bins=config['preprocessing']['mel_bins'],
                fmin=config['preprocessing']['fmin'],
                fmax=config['preprocessing']['fmax'],
                latent_size=config['fit']['latent_size'],
                mixture_size=config['fit']['mixture_size'])
    #pretrained_dict = torch.load(config['IO_OPTION']['PREMODEL_PATH'])
    #net = filtered_load_model(net, pretrained_dict)
    optimizer = optim.Adam(net.parameters(), lr=1e-4)
    num_epochs = config['fit']['num_epochs']
    scheduler = optim.lr_scheduler.OneCycleLR(optimizer=optimizer, pct_start=0.1, div_factor=1e3, 
                                              max_lr=1e-3, epochs=num_epochs, steps_per_epoch=len(dataloaders_dict['train']))
    criterion = nn.MSELoss()
    history = modeler.train_net(net, dataloaders_dict, criterion, optimizer, scheduler, num_epochs, writer)
    # output
    model = history['model']
    model_out_path = MODEL_DIR+'/{}_model.pth'.format(machine_type)
    torch.save(model.state_dict(), model_out_path)
    logger.info('\n success:{0} \n'.format(machine_type) + \
                    'model_out_path ==> \n {0}'.format(model_out_path))
    #  close writer for tensorbord
    writer.close()
    #modeler.mlflow_log(history, config, machine_type, model_out_path, tb_log_dir)
    com.toc()
    return history

In [10]:
machine_types

['fan', 'pump', 'slider', 'ToyCar', 'ToyConveyor', 'valve']

In [11]:
for machine_type in machine_types:
    history = run(machine_type)
    with open('{}/{}_history.pkl'.format(PKL_DIR, machine_type), 'wb') as file:
        pickle.dump(history , file)

2020-11-27 16:23:04,947 - 00_train.py - INFO - TARGET MACHINE_TYPE: fan
2020-11-27 16:23:04,947 - 00_train.py - INFO - MAKE DATA_LOADER
2020-11-27 16:23:05,861 - 00_train.py - INFO - TRAINING


use: cuda:0


100%|██████████| 15/15 [00:02<00:00,  5.32it/s]
2020-11-27 16:23:24,261 - pytorch_modeler.py - INFO - Epoch 1/200:train_loss:1410.715391, tr_rec:18.769588, tr_eng:599.130491, tr_covd:266406.559629, val_AUC:0.500000, val_pAUC:0.500000
100%|██████████| 15/15 [00:02<00:00,  5.26it/s]
2020-11-27 16:23:48,290 - pytorch_modeler.py - INFO - Epoch 2/200:train_loss:90.391304, tr_rec:17.872136, tr_eng:520.698848, tr_covd:4089.856747, val_AUC:0.466590, val_pAUC:0.492511
100%|██████████| 15/15 [00:03<00:00,  4.90it/s]
2020-11-27 16:24:12,468 - pytorch_modeler.py - INFO - Epoch 3/200:train_loss:58.664784, tr_rec:12.446926, tr_eng:332.629471, tr_covd:2590.982311, val_AUC:0.511137, val_pAUC:0.494192
100%|██████████| 15/15 [00:03<00:00,  4.63it/s]
2020-11-27 16:24:37,555 - pytorch_modeler.py - INFO - Epoch 4/200:train_loss:30.884960, tr_rec:5.175920, tr_eng:108.611350, tr_covd:2969.581009, val_AUC:0.512179, val_pAUC:0.501360
100%|██████████| 15/15 [00:03<00:00,  4.88it/s]
2020-11-27 16:25:01,537 - pyt

100%|██████████| 15/15 [00:02<00:00,  5.01it/s]
2020-11-27 16:35:33,673 - pytorch_modeler.py - INFO - Epoch 31/200:train_loss:99.416513, tr_rec:0.904815, tr_eng:39.274783, tr_covd:18916.844193, val_AUC:0.497021, val_pAUC:0.499423
100%|██████████| 15/15 [00:02<00:00,  5.13it/s]
2020-11-27 16:35:57,459 - pytorch_modeler.py - INFO - Epoch 32/200:train_loss:16.313647, tr_rec:0.843456, tr_eng:48.799837, tr_covd:2118.041407, val_AUC:0.488356, val_pAUC:0.499296
100%|██████████| 15/15 [00:03<00:00,  4.82it/s]
2020-11-27 16:36:21,724 - pytorch_modeler.py - INFO - Epoch 33/200:train_loss:25.395815, tr_rec:0.746286, tr_eng:64.968413, tr_covd:3630.537584, val_AUC:0.496966, val_pAUC:0.499839
100%|██████████| 15/15 [00:03<00:00,  4.87it/s]
2020-11-27 16:36:46,556 - pytorch_modeler.py - INFO - Epoch 34/200:train_loss:18.734075, tr_rec:0.565074, tr_eng:70.307289, tr_covd:2227.654484, val_AUC:0.499644, val_pAUC:0.499981
100%|██████████| 15/15 [00:03<00:00,  4.89it/s]
2020-11-27 16:37:10,785 - pytorch_m

RuntimeError: cholesky_cpu: U(6,6) is zero, singular U.

history = run('ToyCar')
with open('{}/{}_history.pkl'.format(PKL_DIR, machine_type), 'wb') as file:
    pickle.dump(history , file)